(1) Generate the data points for all classes $c ∈ {0, . . . C − 1}$, and compute
the approximations $\hat{μ}_c$ and $\hat{Σ}_c$ that are based on the data points. Also,
generate a second set of points, with known tlabels, hat will be used to test
the classification function.

In [ ]:
import numpy as np
import math as m

def getArtificialData(mean, cov, nx, nt):      # define getArtificial function to generate training set, test set, and
    x = np.random.multivariate_normal(mean, cov, size = nx)    # correspondung mean and covarience.
    t = np.random.multivariate_normal(mean, cov, size = nt)

    mu = np.zeros(np.shape(mean))
    for ele in x:
        mu += ele
    mu = mu/nx     # calculating mean of the dataset

    sgm = np.zeros(np.shape(cov))
    for ele in x:
        val = ele - mu
        sgm += np.outer(val, val)
    sgm = sgm/nx                  # calculating varience of the dataset

    return x, t, mu, sgm        # returning the taining set, test set and correspondung mean and covarience.

(2) Evaluate the multivariate Gaussian distribution on the test points of all
classes to obtain the probabilities $p(X = t|Y = c)$, where $t$ runs through
all test points and c runs through all classes.

In [ ]:
def evaluateMultiVarGauss(t, mu, sgm):    # define evaluateMultiVarGauss to generate class conditional density vector of
    r1, r2 = np.shape(t)                 # the training set corresponding to a particular class.
    length = len(t)
    p = np.zeros(length)
    inv_sgm = np.linalg.inv(sgm)     # calculating inverse of the covarience matrix.

    denomerator = pow((2*m.pi), (r2/2)) * pow(np.linalg.det(sgm), 0.5)  # calculating the normalizer factor.
    for i in range(length):
        val = t[i] - mu
        p[i] = np.exp(-(0.5 * np.dot(val, np.matmul(inv_sgm, val))))/denomerator   # calculating the class coditional density of
                                                                        # the training set using multivariate guassian formula.
    return p         # returing the class conditional density vector.

3) Compute the probabilities $p(Y = c|X = t)$ on the test points using Bayes’
formula and for every test point find the value of c with the maximal prob-
ability.

In [ ]:
def calculateTestSet(t, pXY, yEx):     # define calculateTestSet to calculate the number of misclassified item in test set.
    length = len(yEx)                # The columns of pXY matrix contain the class conditional density vector of each class.
    class_label = []

    for ele in yEx:                 # finding how many classes present in the exact level vector.
        if ele in class_label:
            pass
        else:
            class_label.append(ele)

    class_label = np.array(class_label)
    exact = list(yEx)

    prior = []
    for ele in class_label:         # calculating the prior probability vector.
        count1 = exact.count(ele)/length
        prior.append(count1)

    prior = np.array(prior)

    yPredict = []
    yProb = []
    for ele in pXY:             # calculating the predicted label using the baye's theorem.
        denomerator = 0
        for i in range(len(ele)):
            denomerator += prior[i] * ele[i]

        prob = []
        for j in range(len(ele)):
            val = (prior[j] * ele[j])/denomerator
            prob.append(val)

        yProb.append(prob)
        maximum = max(prob)
        ind = prob.index(maximum)
        yPredict.append(ind)

    yProb = np.array(yProb)
    yPredict = np.array(yPredict)

    dummy_merge = np.column_stack((yProb, yEx))   # merging the computed probability matrix, exact class label vector
    merge = np.column_stack((dummy_merge, yPredict))      # and predicted class label vector.

    print('-----:The probability , actual label, predicted label:--------')
    print(merge)

    count_mislabeled = 0        # calculating the number of misclassified item in test set.
    for i in range(length):
        if(yEx[i] != yPredict[i]):
            count_mislabeled += 1

    print('The number of mislabeled points is: ',count_mislabeled )
    return count_mislabeled     # returning the number of misclassified item in test set.

### (1) Run the code as described above.

This will generate quadratic decision boundaries. This should give a very small number of misclassified test points.

### (2) Modify the code to test tied covariances (i.e., linear decision boundaries).

You can compute the tied covariance matrix by the formula.


In [ ]:
mean1 = [0, 0]
cov1 = np.array([[2, 1], [1, 50]])
x1, t1 , mu1, sgm1 = getArtificialData(mean1, cov1, 40, 10)  # calling the getArtificialData() function to calculate
print('The mean for class 0:', mu1)                    # training set, test set, and correspondung mean and covarience for class 0.
print('the varience for class 0:', sgm1)

mean2 = [7, 5]
cov2 = np.array([[3, 1], [1, 3]])
x2, t2 , mu2, sgm2 = getArtificialData(mean2, cov2, 80, 20)  # For class 2 also.
print('The mean for class 1:', mu2)
print('the varience for class 1:', sgm2)

mean3 = [-5, 5]
cov3 = np.array([[5, 2], [2, 3]])
x3, t3 , mu3, sgm3 = getArtificialData(mean3, cov3, 20, 5)  # For class 3 also.
print('The mean for class 2:', mu3)
print('the varience for class 2:', sgm3)

dummy_t = np.concatenate((t1, t2), axis=0)   # getting a general taining set conctenating the taining set of each class.
t = np.concatenate((dummy_t, t3), axis=0)
print('The test data is:')
print(t)

p1 = evaluateMultiVarGauss(t, mu1, sgm1)   # calling evaluateMultiVarGauss() function to generate class conditional density
p2 = evaluateMultiVarGauss(t, mu2, sgm2)   # vector of the training set corresponding to class 0, 1 and 2.
p3 = evaluateMultiVarGauss(t, mu3, sgm3)

dummy_p = np.column_stack((p1, p2))
pXY = np.column_stack((dummy_p, p3))   # defining pXY matrix s.t. the  columns of pXY matrix
                                             # contain the probability vector of each class.
print('The pXY matrix is:')
print(pXY)

a = len(t1)
b = a + len(t2)
c = b + len(t3)
yEx = np.zeros(c)          # calculating the exact class label vector.
yEx[0:a] = 0
yEx[a:b] = 1
yEx[b:c] = 2

misclassified = calculateTestSet(t, pXY, yEx)   # calling calculateTestSet to calculate
                                                     # the number of misclassified item in test set.
error1 = (misclassified / c)*100
print('The percentage of the mislabeled item using actual covarience is:', error1)

print('-----------------------: For tied covarience:----------------------------')

tied_sgm = (len(x1)*sgm1 + len(x2)*sgm2 + len(x3)*sgm3)/(len(x1) + len(x2) + len(x3))    # calculating the tied covarience matrix .
                                             # Then we use above procedure to calculate
print('The tied covarience is:')
print(tied_sgm)

t_p1 = evaluateMultiVarGauss(t, mu1, tied_sgm)                  # the number of misclassified item in test set.
t_p2 = evaluateMultiVarGauss(t, mu2, tied_sgm)
t_p3 = evaluateMultiVarGauss(t, mu3, tied_sgm)

dummy_t_p = np.column_stack((t_p1, t_p2))
t_pXY = np.column_stack((dummy_t_p, t_p3))

print('The pXY matrix is:')
print(t_pXY)

t_yEx = np.zeros(c)
t_yEx[0:a] = 0
t_yEx[a:b] = 1
t_yEx[b:c] = 2

tied_misclassified = calculateTestSet(t, t_pXY, t_yEx)

error2 = (tied_misclassified / c)*100
print('The percentage of the mislabeled item using tied covarience is:', error2)


The mean for class 0: [ 0.13714677 -1.21736209]
the varience for class 0: [[ 1.6638477  -0.70550567]
 [-0.70550567 44.40956715]]
The mean for class 1: [6.97664786 5.10631002]
the varience for class 1: [[3.62058528 0.92852296]
 [0.92852296 3.13754086]]
The mean for class 2: [-5.2820474   4.77830175]
the varience for class 2: [[3.82040382 1.76748075]
 [1.76748075 2.41328905]]
The test data is:
[[ -1.29831523 -19.57906418]
 [  3.90237146   9.67259967]
 [  1.47358114   0.60266003]
 [  0.70065639  -8.93186961]
 [  0.73758355 -15.94157122]
 [ -0.02182711  -6.64534798]
 [ -0.62465728  -2.97692589]
 [  0.36320444  -5.2551286 ]
 [ -1.69295612  -9.38286943]
 [  0.80408558  -7.92020454]
 [  4.10808768   5.38665713]
 [  8.94252997   4.84048531]
 [  6.38426768   3.91215047]
 [  9.45878384   4.93977725]
 [  4.3792163    4.75614592]
 [  8.15105883   5.65495479]
 [  6.11263745   3.14822486]
 [  3.59397407   4.2703036 ]
 [  7.84846373   4.48069296]
 [  5.29747855   1.7588566 ]
 [  8.87520318   7.641533

### 3) Test the methodology on the iris data set, that was mentioned in the lecture.

You can import it with the following commands from sklearn.datasets import load_iris iris = load_iris()

the petal/sepal dimensions are in the array iris.data and the correspond- ing labels are in the array iris.target. You will notice that the first 50 entries have label 0, followed by 50 entries with label 1 and 50 entries with label 2.

Use the first 40 entries in each class as data points and the next 10 entries as test points. You have to write a new routine getIrisData() with similar functionality as getArtificialData(). If you have done the implementation of evaluateMultiVarGauss() and calculateTestSet() right, then you will be able to run these routines with the iris data set as well. Again, count the number of misclassified items in the test set.


In [ ]:
from sklearn.datasets import load_iris       # Importing iris data set using
                                              # load_iris member function of sklearn.datasets module.
def TestSet(x):    # defining TestSet() function to generate the corresponding mean and varience.

    r, c = np.shape(x)
    nx = len(x)
    mu = np.zeros(c)
    for ele in x:
        mu += ele
    mu = mu/nx     # calculating mean of the dataset

    sgm = np.zeros((c, c))
    for ele in x:
        val = ele - mu
        sgm += np.outer(val, val)
    sgm = sgm/nx                  # calculating varience of the dataset

    return mu, sgm

iris = load_iris()
print(type(iris))

feature = iris.data     # extaracting the feature matrix from the iris dataset.
label = iris.target     # extracting class label vector from the iris dataset.

x1 = feature[0:40]    # taking 40 entries for training set of each class label.
x2 = feature[50:90]
x3 = feature[100:140]

t1 = feature[40:50]   # taking next 10 entries for test set of each class label.
t2 = feature[90:100]
t3 = feature[140:150]

dummy_t = np.concatenate((t1, t2), axis=0)   # concatenates the taining sets of each class to obtain the general training set.
t = np.concatenate((dummy_t, t3), axis=0)
print('The test data is:')
print(t)

mu1, sgm1 = TestSet(x1)     # calculating mean and covarience of each class label using TestSet() function.
print('The mean for class 0:', mu1)
print('the varience for class 0:', sgm1)

mu2, sgm2 = TestSet(x2)
print('The mean for class 1:', mu2)
print('the varience for class 1:', sgm2)

mu3, sgm3 = TestSet(x3)
print('The mean for class 2:', mu3)
print('the varience for class 2:', sgm3)

p1 = evaluateMultiVarGauss(t, mu1, sgm1)   # similarly we maintain the same procedure to obtain
p2 = evaluateMultiVarGauss(t, mu2, sgm2)               #  the number of misclassified item in test set.
p3 = evaluateMultiVarGauss(t, mu3, sgm3)

dummy_p = np.column_stack((p1, p2))
pXY = np.column_stack((dummy_p, p3))

print('The pXY matrix is:')
print(pXY)

a = len(t1)
b = a + len(t2)
c = b + len(t3)

yEx = np.zeros(c)
yEx[0:a] = label[40:50]
yEx[a:b] = label[90:100]
yEx[b:c] = label[140: 150]

print('The exact class label vector is:')
print(yEx)

misclassified = calculateTestSet(t, pXY, yEx)

error1 = (misclassified / c)*100
print('The percentage of the mislabeled item using actual covarience is:', error1)


print('------------------------:For tied covarience:-------------------')

tied_sgm = (len(x1)*sgm1 + len(x2)*sgm2 + len(x3)*sgm3)/(len(x1) + len(x2) + len(x3))

print('The tied covarience is:')
print(tied_sgm)

t_p1 = evaluateMultiVarGauss(t, mu1, tied_sgm)
t_p2 = evaluateMultiVarGauss(t, mu2, tied_sgm)
t_p3 = evaluateMultiVarGauss(t, mu3, tied_sgm)

dummy_t_p = np.column_stack((t_p1, t_p2))
t_pXY = np.column_stack((dummy_t_p, t_p3))

print('The pXY matrix is:')
print(t_pXY)

t_yEx = np.zeros(c)
t_yEx[0:a] = label[40:50]
t_yEx[a:b] = label[90:100]
t_yEx[b:c] = label[140: 150]

print('The exact class label vector is:')
print(t_yEx)

tied_misclassified = calculateTestSet(t, t_pXY, t_yEx)

error2 = (tied_misclassified / c)*100
print('The percentage of the mislabeled item using tied covarience is:', error2)


<class 'sklearn.utils._bunch.Bunch'>
The test data is:
[[5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [5.5 2.6 4.4 1.2]
 [6.1 3.  4.6 1.4]
 [5.8 2.6 4.  1.2]
 [5.  2.3 3.3 1. ]
 [5.6 2.7 4.2 1.3]
 [5.7 3.  4.2 1.2]
 [5.7 2.9 4.2 1.3]
 [6.2 2.9 4.3 1.3]
 [5.1 2.5 3.  1.1]
 [5.7 2.8 4.1 1.3]
 [6.7 3.1 5.6 2.4]
 [6.9 3.1 5.1 2.3]
 [5.8 2.7 5.1 1.9]
 [6.8 3.2 5.9 2.3]
 [6.7 3.3 5.7 2.5]
 [6.7 3.  5.2 2.3]
 [6.3 2.5 5.  1.9]
 [6.5 3.  5.2 2. ]
 [6.2 3.4 5.4 2.3]
 [5.9 3.  5.1 1.8]]
The mean for class 0: [5.0375 3.4525 1.46   0.235 ]
the varience for class 0: [[0.12784375 0.09478125 0.013      0.0129375 ]
 [0.09478125 0.12699375 0.0021     0.0116625 ]
 [0.013      0.0021     0.0289     0.0049    ]
 [0.0129375  0.0116625  0.0049     0.009275  ]]
The mean for class 1: [6.01   2.78   4.3175 1.35  ]
the varience for class 1: [[0.2669     0.08445    0.167